# About Dataset
ACME Insurance Inc. offers affordable health insurance to thousands of customer all over the United States. You're tasked with creating an automated system to estimate the annual medical expenditure for new customers, using information such as their age, sex, BMI, children, smoking habits and region of residence.

Estimates from your system will be used to determine the annual insurance premium (amount paid every month) offered to the customer.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/medical-insurance-payout/expenses.csv')
df.head()

# 1 - EDA

In [ ]:
# imports
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Dataset information | Counting Null Values | Datatype Information
df.info()

In [ ]:
# Get categorical and numerics features
feat_cat = df.select_dtypes(include = object).columns
feat_num = df.select_dtypes(exclude = object).columns
feat_cat, feat_num

In [ ]:
# Unique values
unique_values = dict()
for f in feat_cat:
    print(f)
    print(df[f].value_counts().head())
    print('-'*10)
    
    unique_values[f] = list( df[f].unique() )
unique_values

In [ ]:
# Data distribution
## Only BMI has a normal distribution
for f in feat_num:
  sns.displot(data = df, x = f, kind = 'kde', height = 6);

In [ ]:
# Outliers
rows, columns = 2, 2
fig, axes = plt.subplots(rows, columns, figsize=(30, 15))
axes = axes.ravel()
for i in range(rows*columns):
    column = feat_num[i]
    axes[i].boxplot(df[column])
    axes[i].set_title(column)
plt.subplots_adjust(wspace = 0.5)

In [ ]:
# Data Multicollinearity
## age and BMI are somewhat redundant | age e BMI são um pouco redundantes
features = feat_num[:-1] # Exclude target

vif = pd.DataFrame()
vif['feature'] = features
vif["VIF"] = [variance_inflation_factor(df[features].values, i) for i in range(len(features))]
vif

In [ ]:
# Data Correlation
# BMI and AGE have a weak correlation, but are multicollinear | BMI e AGE possuem correlação fraca, mas são multicolineares
fig = plt.figure(figsize = (10, 10))
sns.heatmap(df.corr(), annot = True);

In [ ]:
sns.set_palette('Set2')

In [ ]:
bmi_status = list()
for bmi in df['bmi'].values:
    bmi_status.append( 'underweight' if bmi < 18.5 else 'normal weight' if bmi < 25 else 'overweight' if bmi < 30 else 'obesity' )
df['bmi_status'] = bmi_status
df.head(3)

In [ ]:
plt.figure(figsize = (10, 8))
plt.title('Histograma de fumantes agrupado por sexo')
ax = sns.countplot(x="sex", hue="smoker", data=df);

In [ ]:
plt.figure(figsize = (10, 8))
plt.title('Situação dos fumantes agrupados por BMI')
ax = sns.countplot(x="bmi_status", hue="smoker", data=df);

In [ ]:
plt.figure(figsize = (10, 8))
plt.title('Situação dos fumantes agrupados por quantidade de filhos')
ax = sns.countplot(x="children", hue="smoker", data=df);

In [ ]:
plt.figure(figsize = (10, 8))
plt.title('Histograma de fumantes por região')
sns.histplot(data=df[df['smoker'] == 'yes'], x="region", multiple="dodge", shrink=.8);

In [ ]:
plt.figure(figsize = (10, 8))
plt.title('Histograma de situação coporal')
sns.histplot(data=df, x="bmi_status", multiple="dodge", shrink=.8); # hue="sex",

In [ ]:
plt.figure(figsize = (10, 8))
plt.title('Southeast possui maior índice de fumantes e obesos')
sns.histplot(data=df, x="region", hue="bmi_status", multiple="dodge", shrink=.8);

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=True)
fig.suptitle('Histograma de idade')

# no smoker
sns.histplot(data=df[df['smoker'] == 'no'], x="age", multiple="dodge", shrink=.8, ax=axes[0]);
axes[0].set_title('no smoker');
# smoker
sns.histplot(data=df[df['smoker'] == 'yes'], x="age", multiple="dodge", shrink=.8, ax=axes[1]);
axes[1].set_title('smoker')

# 2 - Data Preparation

In [ ]:
# imports
from category_encoders import LeaveOneOutEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose  import ColumnTransformer

In [ ]:
X, y = df.drop(['charges', 'bmi_status'], axis = 1), df['charges']
X.shape, y.shape

In [ ]:
feat_cat = list( X.select_dtypes(include = object).columns )
feat_num = list( X.select_dtypes(exclude = object).columns )

In [ ]:
## The processes used in the generation of "trees" do not involve the use of distances, therefore, it is not necessary to normalize the numerical data.
## Os processos utilizados para geração de "árvores" não envolvem o uso de distâncias, com isso, não é preciso normalizar os dados numéricos.

numeric_transformer  = Pipeline(steps = [('imputer', SimpleImputer())])
numeric_transformer2 = Pipeline(steps = [('minmax', MinMaxScaler())])
categorical_transformer = Pipeline(steps = [('loo', LeaveOneOutEncoder())])

preprocessor = ColumnTransformer(transformers = [
      ('num', numeric_transformer, feat_num), 
      ('cat', categorical_transformer, feat_cat)]
                                )

preprocessor2 = ColumnTransformer(transformers = [
      ('num', numeric_transformer2, feat_num), 
      ('cat', categorical_transformer, feat_cat)]
                                )

In [ ]:
def get_metrics(y_test, y_pred):
    MAE  = mean_absolute_error(y_test, y_pred)
    MSE  = mean_squared_error(y_test, y_pred)
    RMSE = np.sqrt(MSE)
    R2   = r2_score(y_test, y_pred)
    
    return MAE, MSE, RMSE, R2

In [ ]:
def get_df_pred(y_test, y_pred):
    data = []
    for t, p in zip(y_test.array, y_pred):
      data.append( (int(t), int(p)) )

    return pd.DataFrame(data, columns = ('Test', 'Pred'))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 14)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# imports
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score, make_scorer

# 3 - Algorithms based on decision trees

In [ ]:
# imports
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

## 3.1 - Tuning hiper-parameters

In [ ]:
clf_tree = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", DecisionTreeRegressor())]
)

clf_forest = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestRegressor())]
)

clf_xgbr = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", XGBRegressor())]
)

In [ ]:
cv = KFold(n_splits = 10, shuffle = True, random_state = 14)

### 3.1.1 Decision Tree

In [ ]:
params_tree = {'classifier__criterion': ('squared_error', 'friedman_mse', 'absolute_error', 'poisson'),
              'classifier__splitter': ('best', ),
              'classifier__min_samples_split': (2, 3, 5, 8, 13),
              'classifier__min_samples_leaf': (1, 2, 3, 5, 8),
              'classifier__max_features': ('auto', 'sqrt', 'log2')}

In [ ]:
grid_search_tree = GridSearchCV(clf_tree, scoring = 'r2', param_grid = params_tree, n_jobs =-1, cv = cv, refit = False)
grid_search_tree.fit(X, y)

In [ ]:
results_tree = grid_search_tree.cv_results_
results_tree = pd.DataFrame(data = results_tree, columns = results_tree.keys())
results_tree[results_tree['rank_test_score'] == 1]

### 3.1.2 Random Forest

In [ ]:
params_forest = {'classifier__n_estimators': (21, 34, 55, 89, 100, 144),
                'classifier__criterion': ('squared_error', 'absolute_error', 'poisson'),
                'classifier__min_samples_split': (2, 3, 5, 8),
                'classifier__min_samples_leaf': (1, 2, 3, 5),
                'classifier__max_features': ('sqrt', 'log2'),
                'classifier__oob_score': (True,False)
                }

In [ ]:
grid_search_forest = GridSearchCV(clf_forest, scoring = 'r2', param_grid = params_forest, n_jobs =-1, cv = cv, refit = False)
grid_search_forest.fit(X, y)

In [ ]:
results_forest = grid_search_forest.cv_results_
results_forest = pd.DataFrame(data = results_forest, columns = results_forest.keys())
results_forest[results_forest['rank_test_score'] == 1]

### 3.1.3 XGBooster

In [ ]:
params_xgbr = {'classifier__booster': ('gbtree', ),
               'classifier__sampling_method': ('uniform', 'subsample', 'gradient_based'),
              'classifier__max_depth': (2, 5, 8, 13),
              'classifier__learning_rate': (.1, .3, .5, .8),              
              'classifier__min_child_weight': (1,3,5),
              'classifier__max_delta_step': (0,1,3),
              'classifier__subsample': (.5, 1)}

In [ ]:
grid_search_xgbr = GridSearchCV(clf_xgbr, scoring = 'r2', param_grid = params_xgbr, n_jobs =-1, cv = cv, refit = False, verbose = 1)
grid_search_xgbr.fit(X, y)

In [ ]:
results_xgbr = grid_search_xgbr.cv_results_
results_xgbr = pd.DataFrame(data = results_xgbr, columns = results_xgbr.keys())
results_xgbr[results_xgbr['rank_test_score'] == 1]

## 3.2 Algorithm evaluation
Using the best rated model **(Random Forest)**

In [ ]:
clf_forest = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestRegressor(criterion = 'squared_error',
                                                                                max_features = 'log2',
                                                                                min_samples_leaf = 5,
                                                                                min_samples_split = 8,
                                                                                n_estimators = 55,
                                                                                oob_score = False)
                                           )]
)

In [ ]:
cv    = RepeatedKFold(n_splits = 10, n_repeats = 30, random_state = 14)
score = cross_val_score(clf_forest, X, y, cv=cv, scoring = 'r2', n_jobs = -1)

In [ ]:
print(f'Median R2: {round(score.mean(), 2)} | STD: {round(score.std(), 2)}')

In [ ]:
model_forest = clf_forest.fit(X_train, y_train)
y_pred = model_forest.predict(X_test)

In [ ]:
MAE, MSE, RMSE, R2 = get_metrics(y_test, y_pred)

In [ ]:
print(f'On average, the Random Forest algorithm misses the insurance payment by ${round(MAE, 2)}')
print(f'MSE: {round(MSE, 2)} | RMSE: {round(RMSE, 2)} | R2: {round(R2, 2)}')

In [ ]:
df_pred = get_df_pred(y_test, y_pred)
df_pred.head()

In [ ]:
plt.figure(figsize = (27, 9))
plt.title('Comparison of observed values with predictions')
sns.lineplot(data=df_pred);

# 4 - Algorithms based on linear models

In [ ]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
clf_l2 = Pipeline(
    steps=[("preprocessor", preprocessor2), ("classifier", Ridge())]
)

clf_l1 = Pipeline(
    steps=[("preprocessor", preprocessor2), ("classifier", Lasso())]
)

clf_lin = Pipeline(
    steps=[("preprocessor", preprocessor2), ("classifier", LinearRegression())]
)

clf_poly = Pipeline(
    steps=[("preprocessor", preprocessor2), ('polynomial', PolynomialFeatures(degree = 2)), ("classifier", LinearRegression())]
)

## 4.1 - Ridge Regression

In [ ]:
cv    = RepeatedKFold(n_splits = 10, n_repeats = 30, random_state = 14)
score_l2 = cross_val_score(clf_l2, X, y, cv=cv, scoring = 'r2', n_jobs = -1)

In [ ]:
print(f'Median R2: {round(score_l2.mean(), 2)} | STD: {round(score_l2.std(), 2)} | VAR: {round(score_l2.var(), 5)}')

## 4.2 - Lasso Regression

In [ ]:
cv    = RepeatedKFold(n_splits = 10, n_repeats = 30, random_state = 14)
score_l1 = cross_val_score(clf_l1, X, y, cv=cv, scoring = 'r2', n_jobs = -1)

In [ ]:
print(f'Median R2: {round(score_l1.mean(), 2)} | STD: {round(score_l1.std(), 2)} | VAR: {round(score_l1.var(), 5)}')

## 4.3 - Logistic Regression

In [ ]:
cv    = RepeatedKFold(n_splits = 10, n_repeats = 30, random_state = 14)
score_lin = cross_val_score(clf_lin, X, y, cv=cv, scoring = 'r2', n_jobs = -1)

In [ ]:
print(f'Median R2: {round(score_lin.mean(), 2)} | STD: {round(score_lin.std(), 2)} | VAR: {round(score_lin.var(), 5)}')

## 4.4 Polynomial Regression

In [ ]:
cv    = RepeatedKFold(n_splits = 10, n_repeats = 30, random_state = 14)
score_poly = cross_val_score(clf_poly, X, y, cv=cv, scoring = 'r2', n_jobs = -1)

In [ ]:
print(f'Median R2: {round(score_poly.mean(), 2)} | STD: {round(score_poly.std(), 2)} | VAR: {round(score_poly.var(), 5)}')

## 4.4 Algorithm evaluation
Using the best rated model **(Polynomial Regression)**

In [ ]:
model_poly = clf_poly.fit(X_train, y_train)
y_pred = model_poly.predict(X_test)

In [ ]:
MAE, MSE, RMSE, R2 = get_metrics(y_test, y_pred)

In [ ]:
print(f'On average, the Ridge Regression algorithm misses the insurance payment by ${round(MAE, 2)}')
print(f'MSE: {round(MSE, 2)} | RMSE: {round(RMSE, 2)} | R2: {round(R2, 2)}')

In [ ]:
df_pred = get_df_pred(y_test, y_pred)
df_pred.head()

In [ ]:
plt.figure(figsize = (30, 9))
plt.title('Comparison of observed values with predictions')
sns.lineplot(data=df_pred);

In [ ]:
# Residuals Plot
from yellowbrick.regressor import ResidualsPlot
visualizer = ResidualsPlot(model_poly)

visualizer.fit(X_train, y_train)  
visualizer.score(X_test, y_test)  
visualizer.show();              